# Dr. Simonson Chronic Mountain Sickness 

# RNASeq Data Integration

   > * Art Nasamran, CCBB (cnasamran@ucsd.edu)
   > * Based on upstream analysis by Guorong Xu, CCBB (g1xu@ucsd.edu)

* Modeled on "RNA-seq analysis is easy as 1-2-3 with limma, Glimma and edgeR" ([1](#Citations))

## Table of Contents
* [Background](#Background)
* [Introduction](#Introduction)
* [Parameter Input](#Parameter-Input)
* [Library Import](#Library-Import)
* [Data Import](#Data-Import)
    * [Count Data](#Count-Data)
    * [Metadata](#Metadata)
    * [Annotations](#Annotations)
* [Gene Separation By Coding Status](#Gene-Separation-By-Coding-Status)
* [Data Integration](#Data-Integration)
* [Annotation Integration](#Annotation-Integration)
* [Summary](#Summary)
* [Citations](#Citations)
* [Appendix: R Session Info](#Appendix:-R-Session-Info)


## Background

The count data analyzed in this notebook were produced by the primary analysis of Dr. Guorong Xu of CCBB, who received raw sequencing data and performed quality control (Fastqc v0.10.0),  alignment (STAR v2.5.3), and quantification (RSEMv1.3.0) of reads using GRCh38.p12.genome.fa and gencode.v29.annotation.gtf.


[Table of Contents](#Table-of-Contents)

## Introduction

This notebook takes in per-gene-per-sample count data (prepared either externally or by the  "RNASeq_RSEM_QC_and_Counts_Preparation" notebook) and per-sample metadata RNASeq data, and uses the edgeR ([2](#Citations)) Bioconductor ([3](#Citations)) package written in R ([4](#Citations)) to integrate and annotate these inputs in preparation for data exploration and preprocessing.

[Table of Contents](#Table-of-Contents)

## Parameter Input

In [1]:
gProjectName = "20191108_simonson_rnaseq"
gGeneCountsFilename = "all_gene_counts.txt"
gMetadataFilename = "metadata_20200203.txt"
# Needed to remove "rowname" from "all_gene_counts.txt"

# Note: the below R object file is necessary only if separating genes into coding 
# and non-coding before analysis.
# If NOT doing this, set the value of this filename to NULL.
# Note: be sure to record source of annotations file, such as:
# R object of contents of gencode.v29.annotation.gtf based on Homo sapiens GRCh38p12
gAnnotationsRdataFilename = "Homo_sapiens_GRCh38p12_gencodev29_ANNOT.Rdata"

In [2]:
gSourceDir = "./src/" # note trailing slash here but not below
gOutputDir = "../outputs"
gReferenceDir = "../reference"
gInterimDir = "../interim"
gGeneCountsFp = file.path(gOutputDir, gGeneCountsFilename)
gMetadataFp = file.path(gReferenceDir, gMetadataFilename)

In [3]:
# Import shared source code to load and save previous notebooks' environments:
source(paste0(gSourceDir, "ChainedNotebookSupport.R"))

Populate the run name parameter automatically to ensure that outputs from different runs do not overwrite each other:

In [4]:
gRunName = makeRunName(gProjectName, "data_integration")
gRunName

[1] "20191108_simonson_rnaseq_data_integration_20200213120626"

[Table of Contents](#Table-of-Contents)

## Library Import

Import the necessary R, Bioconductor, and CCBB libraries for the analysis:

In [ ]:
#if (!requireNamespace("BiocManager", quietly = TRUE))
#    install.packages("BiocManager")

In [ ]:
#BiocManager::install("edgeR", version = "3.8")

In [ ]:
#BiocManager::install("Homo.sapiens", version = "3.8")

In [5]:
library(Homo.sapiens)
gOrganismPackage = Homo.sapiens

Loading required package: AnnotationDbi
Loading required package: stats4
Loading required package: BiocGenerics
Loading required package: parallel

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB

The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs

The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colMeans,
    colnames, colSums, dirname, do.call, duplicated, eval, evalq,
    Filter, Find, get, grep, grepl, intersect, is.unsorted, lapply,
    lengths, Map, mapply, match, mget, order, paste, pmax, pmax.int,
    pmin, pmin.int, Position, rank, rbind, Reduce, rowMeans, rownames,
    rowSums, sapply, setdiff, sort, table, tapply, union, unique,
    unsplit, which, which.max, w

In [6]:
library(edgeR)

Loading required package: limma

Attaching package: ‘limma’

The following object is masked from ‘package:BiocGenerics’:

    plotMA



[Table of Contents](#Table-of-Contents)


## Data Import

### Count Data

Import the count data file in which rows are genes identifiers, columns are sample identifiers, and row/column intersections contain the number of counts for the relevant gene in the relevant sample:

In [7]:
# Read in counts file containing info on all samples and genes
gUnorderedGeneCountsDf <- read.csv(gGeneCountsFp, sep="\t", stringsAsFactors=FALSE, row.names=1)
dim(gUnorderedGeneCountsDf)

[1] 58676    65

In [8]:
head(gUnorderedGeneCountsDf)

,EL20_S20_L004_R1_001,EL62_S62_L004_R1_001,EL60_S60_L004_R1_001,EL59_S59_L004_R1_001,EL61_S61_L004_R1_001,EL29_S29_L004_R1_001,EL49_S49_L004_R1_001,EL32_S32_L004_R1_001,EL51_S51_L004_R1_001,EL22_S22_L004_R1_001,⋯,EL6_S6_L004_R1_001,EL24_S24_L004_R1_001,EL10_S10_L004_R1_001,EL12_S12_L004_R1_001,EL34_S34_L004_R1_001,EL38_S38_L004_R1_001,EL58_S58_L004_R1_001,EL50_S50_L004_R1_001,EL44_S44_L004_R1_001,EL63_S63_L004_R1_001
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ENSG00000000003.14,1.00,2.00,2.00,4.00,4.00,0.00,3.00,2.00,2.00,4.00,⋯,1.00,3.00,2.00,0.00,1.00,4.00,3.00,0.00,4.00,4.00
ENSG00000000005.5,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,⋯,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
ENSG00000000419.12,118.00,127.00,107.00,159.00,79.00,83.00,100.00,151.00,189.00,79.00,⋯,47.00,118.00,92.00,64.00,183.00,189.00,86.00,137.00,184.00,118.00
ENSG00000000457.13,240.62,276.31,254.35,225.02,234.61,200.96,165.91,269.95,323.61,157.46,⋯,106.28,294.79,227.07,203.26,263.25,272.84,186.32,216.41,292.29,253.39
ENSG00000000460.16,88.38,68.69,50.65,75.98,65.39,55.04,54.09,53.05,87.39,42.54,⋯,31.72,57.21,56.93,56.74,72.75,66.16,71.68,46.59,86.71,57.61
ENSG00000000938.12,2142.00,2545.00,2243.00,2249.00,1732.00,2083.00,2114.00,2422.00,3041.00,1814.00,⋯,754.00,2276.00,2138.00,2273.00,2817.00,2381.00,1847.00,1936.00,2874.00,2931.00


No assumption is made that the columns (samples) of the gene count file are currently ordered in the order desirable for the differential expression analysis.

[Table of Contents](#Table-of-Contents)

### Metadata

> For downstream analysis, sample-level information related to the experimental design needs to be associated with the columns of the counts matrix. This should include experimental variables, both biological and technical, that could have an effect on expression levels. Examples [could] include cell type (basal, LP and ML in this experiment), genotype (wild-type, knock-out), phenotype (disease status, sex, age), sample treatment (drug, control) and batch information (date experiment was performed if samples were collected and analysed at distinct time points) to name just a few. ([1](#Citations))

Import a metadata file in which rows are sample identifiers, columns are metadata features (e.g., subject id, time point, etc) and row/column intersections contain the value of the relevant feature for the relevant sample:

In [9]:
#Read in metadata
gMetadataDf <- read.csv(gMetadataFp, stringsAsFactors=FALSE, sep = "\t")
dim(gMetadataDf)

[1] 65 13

In [10]:
head(gMetadataDf)

File_Name,Sample_name,Sample_ID,Patient,Condition,Treatment,Condition_Treatment,Tissue,Date_Collected,Physio_State,Treatment.1,Pass_QC,cpf_include
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>
EL24_S24_L004_R1_001.fastq.gz,EL24_S24_L004_R1_001,S24,MAN_4,CMS,POSTHEM_FASTING,CMS_POSTHEM_FASTING,Whole Blood,15-Dec-17,POSTHEM,FASTING,,1
EL54_S54_L004_R1_001.fastq.gz,EL54_S54_L004_R1_001,S54,MAN_4,CMS,POSTHEM_FASTING,CMS_POSTHEM_FASTING,Whole Blood,13-Dec-17,POSTHEM,FASTING,,NA
EL27_S27_L004_R1_001.fastq.gz,EL27_S27_L004_R1_001,S27,MVT_6,CMS,POSTHEM_FASTING,CMS_POSTHEM_FASTING,Whole Blood,14-Dec-17,POSTHEM,FASTING,,NA
EL52_S52_L004_R1_001.fastq.gz,EL52_S52_L004_R1_001,S52,MVT_6,CMS,POSTHEM_FASTING,CMS_POSTHEM_FASTING,Whole Blood,15-Dec-17,POSTHEM,FASTING,,1
EL26_S26_L004_R1_001.fastq.gz,EL26_S26_L004_R1_001,S26,NCR_3,CMS,POSTHEM_FASTING,CMS_POSTHEM_FASTING,Whole Blood,13-Dec-17,POSTHEM,FASTING,,NA
EL41_S41_L004_R1_001.fastq.gz,EL41_S41_L004_R1_001,S41,NCR_3,CMS,POSTHEM_FASTING,CMS_POSTHEM_FASTING,Whole Blood,15-Dec-17,POSTHEM,FASTING,,1


In [11]:
gSampleNames = gMetadataDf[["Sample_name"]]

Check the dimensions of the count data and the metadata to ensure that the count dataframe has the same number of columns (samples) as the metadata dataframe has rows (again, samples), and that the sample names are the same in both: 

In [12]:
dim(gUnorderedGeneCountsDf)
dim(gMetadataDf)

all(colnames(gUnorderedGeneCountsDf) %in% gSampleNames)

[1] 58676    65

[1] 65 13

[1] TRUE

Assume that the order of the samples shown in the metadata is the desired order, and reorder the columns in the counts table to match it:

In [14]:
gGeneCountsDf = gUnorderedGeneCountsDf[gSampleNames]
head(gGeneCountsDf)

,EL24_S24_L004_R1_001,EL54_S54_L004_R1_001,EL27_S27_L004_R1_001,EL52_S52_L004_R1_001,EL26_S26_L004_R1_001,EL41_S41_L004_R1_001,EL2_S2_L004_R1_001,EL34_S34_L004_R1_001,EL25_S25_L004_R1_001,EL62_S62_L004_R1_001,⋯,EL48_S48_L004_R1_001,EL63_S63_L004_R1_001,EL16_S16_L004_R1_001,EL40_S40_L004_R1_001,EL11_S11_L004_R1_001,EL37_S37_L004_R1_001,EL12_S12_L004_R1_001,EL44_S44_L004_R1_001,EL23_S23_L004_R1_001,EL51_S51_L004_R1_001
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ENSG00000000003.14,3.00,3.00,0.00,0.00,2.00,1.00,1.00,1.00,2.00,2.00,⋯,3.00,4.00,1.00,1.0,2.00,3.00,0.00,4.00,4.00,2.00
ENSG00000000005.5,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,⋯,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00
ENSG00000000419.12,118.00,87.00,88.00,52.00,108.00,122.00,45.00,183.00,63.00,127.00,⋯,179.00,118.00,55.00,170.0,138.00,147.00,64.00,184.00,125.00,189.00
ENSG00000000457.13,294.79,220.08,316.65,214.03,228.57,167.65,123.35,263.25,139.32,276.31,⋯,354.97,253.39,218.78,330.3,364.02,307.78,203.26,292.29,314.67,323.61
ENSG00000000460.16,57.21,58.92,112.35,73.97,64.43,61.35,35.65,72.75,29.68,68.69,⋯,93.03,57.61,52.22,54.7,82.98,77.22,56.74,86.71,102.33,87.39
ENSG00000000938.12,2276.00,1860.00,2783.00,1903.00,2381.00,1410.00,1593.00,2817.00,1454.00,2545.00,⋯,2880.00,2931.00,2326.00,2224.0,2855.00,2363.00,2273.00,2874.00,3236.00,3041.00


If the count file gene identifiers do NOT include version numbers (e.g., the ".4" part in a gene identifier like "ENSG00000268020.4"), then it is necessary to truncate the version information from the public annotation data to be used below in order to match the annotation data gene identifiers to the count file gene identifiers.  Set the flag for version removal accordingly:

In [15]:
gRemoveVersion <- FALSE
#gRemoveVersion <- TRUE

[Table of Contents](#Table-of-Contents)

###  Annotations

If a previously created file of the gene annotations has been provided, load it:

In [16]:

if (!is.null(gAnnotationsRdataFilename)) {
    gAnnotationsRdataFp = file.path(gReferenceDir, gAnnotationsRdataFilename)  
    
    # Import the R data object containing gene annotations and load its dataframe into a variable:
    gAnnotationEnv = loadToEnvironment(gAnnotationsRdataFp)
    gGeneTypeAnnotationsDf = gAnnotationEnv$ANNOT
    
    head(gGeneTypeAnnotationsDf)
} else {
    print("No annotations provided.")
}

gene_type,gene_id,transcript_id
<chr>,<chr>,<chr>
transcribed_unprocessed_pseudogene,ENSG00000223972.5,ENST00000456328.2
transcribed_unprocessed_pseudogene,ENSG00000223972.5,ENST00000450305.2
unprocessed_pseudogene,ENSG00000227232.5,ENST00000488147.1
miRNA,ENSG00000278267.1,ENST00000619216.1
lincRNA,ENSG00000243485.5,ENST00000473358.1
lincRNA,ENSG00000243485.5,ENST00000469289.1


[Table of Contents](#Table-of-Contents)

## Gene Separation By Coding Status

Gene annotations are records of each gene's identifier and symbol, where the gene begins and ends on the genome sequence, and whether it is anticipated to be a coding gene or not.  There are multiple sources of gene annotations.


   > Here we use the human gene annotations from the Gencode project, Release 29 (GRCh38.p12).


In [17]:
splitGeneCountsByCodingStatus = function(geneCountDf, gtfDf, removeVersion=FALSE){
    #Subset GTF by protein coding and noncoding
    ANNOT_protein_coding <- subset(gtfDf, gene_type == "protein_coding")
    ANNOT_ncRNA <- subset(gtfDf, gene_type %in% c("lincRNA", "antisense", "processed_transcript","sense_overlapping", "sense_intronic") )

    #make list of IDs to query
    protein_coding_ids <- ANNOT_protein_coding$gene_id
    ncRNA_ids <- ANNOT_ncRNA$gene_id
    
    if (removeVersion){
        protein_coding_ids <- removeAccessionVersion(protein_coding_ids)
        ncRNA_ids <- removeAccessionVersion(ncRNA_ids)        
    }

    #subset geneCounts
    geneCount_protein_coding <- subset(geneCountDf, row.names(geneCountDf) %in% protein_coding_ids)
    geneCount_ncRNA <- subset(geneCountDf, row.names(geneCountDf) %in% ncRNA_ids)
    return(list(codingGeneCountDf=geneCount_protein_coding, noncodingGeneCountDf=geneCount_ncRNA))
}

removeAccessionVersion = function(accessionVector){
    return (gsub("\\..*","",accessionVector))
}

writeSubsetCounts = function(subsetCountsDf, outputDir, runName, fileSuffix){
    fileName = sprintf(fileSuffix, runName)
    write.csv(subsetCountsDf, file.path(outputDir, fileName))
    print(paste0("Output file: ",fileName))
}

writeSubsetsCounts = function(splitGeneCountDfsList, outputDir, runName){
    writeSubsetCounts(splitGeneCountDfsList$codingGeneCountDf, outputDir, runName,"%s_raw_pc_genes_counts.csv")
    writeSubsetCounts(splitGeneCountDfsList$noncodingGeneCountDf, outputDir, runName,"%s_raw_nc_genes_counts.csv")
}

Split the count data into coding and non-coding subsets, and extract each subset into a file based on the annotation file provided in the input parameters:

In [18]:
gSplitGeneCountDfsList = splitGeneCountsByCodingStatus(gGeneCountsDf, gGeneTypeAnnotationsDf, gRemoveVersion)

In [19]:
dim(gGeneCountsDf)
dim(gSplitGeneCountDfsList$codingGeneCountDf)
dim(gSplitGeneCountDfsList$noncodingGeneCountDf)

[1] 58676    65

[1] 19922    65

[1] 14886    65

In [20]:
writeSubsetsCounts(gSplitGeneCountDfsList, gOutputDir, gRunName)

[1] "Output file: 20191108_simonson_rnaseq_data_integration_20200213120626_raw_pc_genes_counts.csv"
[1] "Output file: 20191108_simonson_rnaseq_data_integration_20200213120626_raw_nc_genes_counts.csv"


[Table of Contents](#Table-of-Contents)

## Data Integration



Integrate the count data and the metadata into an edgeR DGEList object for use in downstream analysis:

> Our DGEList-object contains a samples data frame that stores both ... group ... and batch ... information, each of which consists of ... distinct levels. Note that within x$samples, library sizes are automatically calculated for each sample and normalisation factors are set to 1. ([1](#Citations))

In [21]:
# If the researcher wishes to analyze just coding genes, run: 
gGeneType = "pc"
gRelevantGeneCountsDf <- gSplitGeneCountDfsList$codingGeneCountDf

# If researcher wishes to analyze just non-coding genes, run:
# gGeneType = "nc"
# gRelevantGeneCountsDf <- gSplitGeneCountDfsList$noncodingGeneCountDf

# If researcher wishes to analyze both coding and non-coding genes together, run:
# gGeneType = "all"
# gRelevantGeneCountsDf <- gGeneCountsDf

In [22]:
# create a DGEList object
makeDgeList = function(countsDf, metadataDf, groupColName){
    # remove the accession version (.##etc) from the ensembl gene id
    id_list <- gsub("[.].*$","", row.names(countsDf))
    row.names(countsDf) <- id_list
    # Note: in DGEList constructor, parameters
    # lib.size = colSums(counts_matrix), 
    # norm.factors = rep(1,ncol(counts_matrix)), 
    # genes = NULL, and remove.zeros = FALSE
    # are all identical to the default values you'd get if you didn't 
    # specify these arguments at all ...
    x <- DGEList(counts = countsDf, lib.size = colSums(countsDf),
    norm.factors = rep(1,ncol(countsDf)), samples = metadataDf,
        group = metadataDf[[groupColName]], genes = NULL, remove.zeros = FALSE)
    return(x)
}

In [23]:
gGroupCategory = "Condition_Treatment"

In [24]:
gDgeList = makeDgeList(gRelevantGeneCountsDf, gMetadataDf, gGroupCategory)
names(gDgeList)

[1] "counts"  "samples"

As a sanity-check, look at representative content from the DGEList:

In [25]:
head(gDgeList$counts)
head(gDgeList$samples)

,EL24_S24_L004_R1_001,EL54_S54_L004_R1_001,EL27_S27_L004_R1_001,EL52_S52_L004_R1_001,EL26_S26_L004_R1_001,EL41_S41_L004_R1_001,EL2_S2_L004_R1_001,EL34_S34_L004_R1_001,EL25_S25_L004_R1_001,EL62_S62_L004_R1_001,⋯,EL48_S48_L004_R1_001,EL63_S63_L004_R1_001,EL16_S16_L004_R1_001,EL40_S40_L004_R1_001,EL11_S11_L004_R1_001,EL37_S37_L004_R1_001,EL12_S12_L004_R1_001,EL44_S44_L004_R1_001,EL23_S23_L004_R1_001,EL51_S51_L004_R1_001
ENSG00000000003,3.00,3.00,0.00,0.00,2.00,1.00,1.00,1.00,2.00,2.00,⋯,3.00,4.00,1.00,1.0,2.00,3.00,0.00,4.00,4.00,2.00
ENSG00000000005,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,⋯,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00
ENSG00000000419,118.00,87.00,88.00,52.00,108.00,122.00,45.00,183.00,63.00,127.00,⋯,179.00,118.00,55.00,170.0,138.00,147.00,64.00,184.00,125.00,189.00
ENSG00000000457,294.79,220.08,316.65,214.03,228.57,167.65,123.35,263.25,139.32,276.31,⋯,354.97,253.39,218.78,330.3,364.02,307.78,203.26,292.29,314.67,323.61
ENSG00000000460,57.21,58.92,112.35,73.97,64.43,61.35,35.65,72.75,29.68,68.69,⋯,93.03,57.61,52.22,54.7,82.98,77.22,56.74,86.71,102.33,87.39
ENSG00000000938,2276.00,1860.00,2783.00,1903.00,2381.00,1410.00,1593.00,2817.00,1454.00,2545.00,⋯,2880.00,2931.00,2326.00,2224.0,2855.00,2363.00,2273.00,2874.00,3236.00,3041.00


,group,lib.size,norm.factors,File_Name,Sample_name,Sample_ID,Patient,Condition,Treatment,Condition_Treatment,Tissue,Date_Collected,Physio_State,Treatment.1,Pass_QC,cpf_include
,<fct>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>
EL24_S24_L004_R1_001,CMS_POSTHEM_FASTING,10664912,1,EL24_S24_L004_R1_001.fastq.gz,EL24_S24_L004_R1_001,S24,MAN_4,CMS,POSTHEM_FASTING,CMS_POSTHEM_FASTING,Whole Blood,15-Dec-17,POSTHEM,FASTING,,1
EL54_S54_L004_R1_001,CMS_POSTHEM_FASTING,9374743,1,EL54_S54_L004_R1_001.fastq.gz,EL54_S54_L004_R1_001,S54,MAN_4,CMS,POSTHEM_FASTING,CMS_POSTHEM_FASTING,Whole Blood,13-Dec-17,POSTHEM,FASTING,,NA
EL27_S27_L004_R1_001,CMS_POSTHEM_FASTING,10723006,1,EL27_S27_L004_R1_001.fastq.gz,EL27_S27_L004_R1_001,S27,MVT_6,CMS,POSTHEM_FASTING,CMS_POSTHEM_FASTING,Whole Blood,14-Dec-17,POSTHEM,FASTING,,NA
EL52_S52_L004_R1_001,CMS_POSTHEM_FASTING,8225898,1,EL52_S52_L004_R1_001.fastq.gz,EL52_S52_L004_R1_001,S52,MVT_6,CMS,POSTHEM_FASTING,CMS_POSTHEM_FASTING,Whole Blood,15-Dec-17,POSTHEM,FASTING,,1
EL26_S26_L004_R1_001,CMS_POSTHEM_FASTING,9867201,1,EL26_S26_L004_R1_001.fastq.gz,EL26_S26_L004_R1_001,S26,NCR_3,CMS,POSTHEM_FASTING,CMS_POSTHEM_FASTING,Whole Blood,13-Dec-17,POSTHEM,FASTING,,NA
EL41_S41_L004_R1_001,CMS_POSTHEM_FASTING,9499867,1,EL41_S41_L004_R1_001.fastq.gz,EL41_S41_L004_R1_001,S41,NCR_3,CMS,POSTHEM_FASTING,CMS_POSTHEM_FASTING,Whole Blood,15-Dec-17,POSTHEM,FASTING,,1


[Table of Contents](#Table-of-Contents)

## Annotation Integration

Next, extend the DGEList object with annotation information about the genes that have count data with symbol and EntrezId information, based upon their Ensembl ids.

> A second data frame named genes in the DGEList-object is used to store gene-level information associated with rows of the counts matrix. This information can be retrieved using organism specific packages such as Mus.musculus (Bioconductor Core Team 2016b) for mouse (or Homo.sapiens (Bioconductor Core Team 2016a) for human) ....
>
> The type of information that can be retrieved includes gene symbols, gene names, chromosome names and locations, Entrez gene IDs, Refseq gene IDs and Ensembl gene IDs to name just a few. .... Mus.musculus [and other organism-specific packages] packages information from various sources and allows users to choose between many different gene IDs as the key. ([1](#Citations))

In [26]:
getGeneDf = function(dgeList, organismPackage){
    geneid <-  rownames(dgeList)
    genes <- select(organismPackage, keys=geneid, columns=c("SYMBOL", "ENSEMBL", "ENTREZID"), 
                    keytype="ENSEMBL")
    return(genes)
}

In [27]:
gRawGenesDf = getGeneDf(gDgeList, gOrganismPackage)
dim(gRawGenesDf)

'select()' returned 1:many mapping between keys and columns


[1] 20190     3

In [28]:
# Add gene type to gRawGenesDf
gGeneTypeAnnotationsDf.rmdec <- gGeneTypeAnnotationsDf
gGeneTypeAnnotationsDf.rmdec$gene_id <- gsub("\\..*","",gGeneTypeAnnotationsDf.rmdec$gene_id)
gRawGenesDf$gene_type <- gGeneTypeAnnotationsDf$gene_type[match(gRawGenesDf$ENSEMBL, gGeneTypeAnnotationsDf.rmdec$gene_id)]

In [29]:
head(gRawGenesDf)

ENSEMBL,ENTREZID,SYMBOL,gene_type
<chr>,<chr>,<chr>,<chr>
ENSG00000000003,7105,TSPAN6,protein_coding
ENSG00000000005,64102,TNMD,protein_coding
ENSG00000000419,8813,DPM1,protein_coding
ENSG00000000457,57147,SCYL3,protein_coding
ENSG00000000460,55732,C1orf112,protein_coding
ENSG00000000938,2268,FGR,protein_coding


> [G]ene IDs may not map one-to-one to the gene information of interest. It is important to check for duplicated gene IDs. ([1](#Citations))

Examine how many records in the annotation dataset have the same id (for the gene identifier type--either ENSEMBL or ENTREZ--set below) as another record occurring earlier in the dataset:

In [30]:
gGeneIdCol <- "ENSEMBL"
# gGeneIdCol <- "ENTREZ"

In [31]:
gDuplicatesMask = duplicated(gRawGenesDf[[gGeneIdCol]])
sum(gDuplicatesMask) # Sum counts only those with a value of TRUE

[1] 268

Note that this sum includes only the second (or greater) instances of records for each gene id; the first record for each gene id is not included in this duplicate set.

Write a file of the duplicate records that can be examined if desired: 

In [32]:
writeOutRemovedDuplicates = function(countsDf, duplicatesMask, outputDir, runName, geneType){
    fileName = sprintf("%s_duplicated_%s_genes_records.csv",runName, geneType)
    duplicatedCountsDf = countsDf[duplicatesMask,]
    write.csv(duplicatedCountsDf, file.path(outputDir, fileName))
    print(paste0("Output file: ",fileName))
}

In [33]:
writeOutRemovedDuplicates(gRawGenesDf, gDuplicatesMask, gOutputDir, gRunName, gGeneType)

[1] "Output file: 20191108_simonson_rnaseq_data_integration_20200213120626_duplicated_pc_genes_records.csv"


In [34]:
gDeduplicatedGenesDf = gRawGenesDf[!duplicated(gRawGenesDf[[gGeneIdCol]]),]

After deduplication, check the dimensions of the count data and the gene annotation data to ensure that the count dataframe has the same number of rows (genes) as the gene annotation dataframe has rows (again, genes), and that the gene names are the same in both:

In [35]:
dim(gDgeList$counts)
dim(gDeduplicatedGenesDf)

all(rownames(gDgeList$counts) %in% gDeduplicatedGenesDf[[gGeneIdCol]])

[1] 19922    65

[1] 19922     4

[1] TRUE

Add the annotation information to the DGEList object:

In [36]:
gDgeList$genes = gDeduplicatedGenesDf
names(gDgeList)

[1] "counts"  "samples" "genes"

As a sanity-check, look at representative content from the DGEList:

In [37]:
head(gDgeList$counts)
head(gDgeList$samples)
head(gDgeList$genes)

,EL24_S24_L004_R1_001,EL54_S54_L004_R1_001,EL27_S27_L004_R1_001,EL52_S52_L004_R1_001,EL26_S26_L004_R1_001,EL41_S41_L004_R1_001,EL2_S2_L004_R1_001,EL34_S34_L004_R1_001,EL25_S25_L004_R1_001,EL62_S62_L004_R1_001,⋯,EL48_S48_L004_R1_001,EL63_S63_L004_R1_001,EL16_S16_L004_R1_001,EL40_S40_L004_R1_001,EL11_S11_L004_R1_001,EL37_S37_L004_R1_001,EL12_S12_L004_R1_001,EL44_S44_L004_R1_001,EL23_S23_L004_R1_001,EL51_S51_L004_R1_001
ENSG00000000003,3.00,3.00,0.00,0.00,2.00,1.00,1.00,1.00,2.00,2.00,⋯,3.00,4.00,1.00,1.0,2.00,3.00,0.00,4.00,4.00,2.00
ENSG00000000005,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,⋯,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00
ENSG00000000419,118.00,87.00,88.00,52.00,108.00,122.00,45.00,183.00,63.00,127.00,⋯,179.00,118.00,55.00,170.0,138.00,147.00,64.00,184.00,125.00,189.00
ENSG00000000457,294.79,220.08,316.65,214.03,228.57,167.65,123.35,263.25,139.32,276.31,⋯,354.97,253.39,218.78,330.3,364.02,307.78,203.26,292.29,314.67,323.61
ENSG00000000460,57.21,58.92,112.35,73.97,64.43,61.35,35.65,72.75,29.68,68.69,⋯,93.03,57.61,52.22,54.7,82.98,77.22,56.74,86.71,102.33,87.39
ENSG00000000938,2276.00,1860.00,2783.00,1903.00,2381.00,1410.00,1593.00,2817.00,1454.00,2545.00,⋯,2880.00,2931.00,2326.00,2224.0,2855.00,2363.00,2273.00,2874.00,3236.00,3041.00


,group,lib.size,norm.factors,File_Name,Sample_name,Sample_ID,Patient,Condition,Treatment,Condition_Treatment,Tissue,Date_Collected,Physio_State,Treatment.1,Pass_QC,cpf_include
,<fct>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>
EL24_S24_L004_R1_001,CMS_POSTHEM_FASTING,10664912,1,EL24_S24_L004_R1_001.fastq.gz,EL24_S24_L004_R1_001,S24,MAN_4,CMS,POSTHEM_FASTING,CMS_POSTHEM_FASTING,Whole Blood,15-Dec-17,POSTHEM,FASTING,,1
EL54_S54_L004_R1_001,CMS_POSTHEM_FASTING,9374743,1,EL54_S54_L004_R1_001.fastq.gz,EL54_S54_L004_R1_001,S54,MAN_4,CMS,POSTHEM_FASTING,CMS_POSTHEM_FASTING,Whole Blood,13-Dec-17,POSTHEM,FASTING,,NA
EL27_S27_L004_R1_001,CMS_POSTHEM_FASTING,10723006,1,EL27_S27_L004_R1_001.fastq.gz,EL27_S27_L004_R1_001,S27,MVT_6,CMS,POSTHEM_FASTING,CMS_POSTHEM_FASTING,Whole Blood,14-Dec-17,POSTHEM,FASTING,,NA
EL52_S52_L004_R1_001,CMS_POSTHEM_FASTING,8225898,1,EL52_S52_L004_R1_001.fastq.gz,EL52_S52_L004_R1_001,S52,MVT_6,CMS,POSTHEM_FASTING,CMS_POSTHEM_FASTING,Whole Blood,15-Dec-17,POSTHEM,FASTING,,1
EL26_S26_L004_R1_001,CMS_POSTHEM_FASTING,9867201,1,EL26_S26_L004_R1_001.fastq.gz,EL26_S26_L004_R1_001,S26,NCR_3,CMS,POSTHEM_FASTING,CMS_POSTHEM_FASTING,Whole Blood,13-Dec-17,POSTHEM,FASTING,,NA
EL41_S41_L004_R1_001,CMS_POSTHEM_FASTING,9499867,1,EL41_S41_L004_R1_001.fastq.gz,EL41_S41_L004_R1_001,S41,NCR_3,CMS,POSTHEM_FASTING,CMS_POSTHEM_FASTING,Whole Blood,15-Dec-17,POSTHEM,FASTING,,1


ENSEMBL,ENTREZID,SYMBOL,gene_type
<chr>,<chr>,<chr>,<chr>
ENSG00000000003,7105,TSPAN6,protein_coding
ENSG00000000005,64102,TNMD,protein_coding
ENSG00000000419,8813,DPM1,protein_coding
ENSG00000000457,57147,SCYL3,protein_coding
ENSG00000000460,55732,C1orf112,protein_coding
ENSG00000000938,2268,FGR,protein_coding


[Table of Contents](#Table-of-Contents)

## Summary


> **Gene annotations**
* Human gene annotations were taken from the Gencode project, Release 29 (GRCh38.p12).

> **Gene type filtering**
* This analysis was limited to protein-coding genes. Of the original 58676 Ensembl genes in the dataset, 19922 are known coding genes.

Save the workspace objects for future reference:

In [38]:
writeWorkspaceImage(gInterimDir, gRunName)

[1] "Output file: 20191108_simonson_rnaseq_data_integration_20200213120626.RData"


[Table of Contents](#Table-of-Contents)

## Citations

1. Law CW, Alhamdoosh M, Su S, Smyth GK, Ritchie ME. RNA-seq analysis is easy as 1-2-3 with limma, Glimma and edgeR. Version 2. F1000Res. 2016 Jun 17 [revised 2016 Jan 1];5:1408.
2. Robinson MD, McCarthy DJ and Smyth GK (2010). edgeR: a Bioconductor package for differential expression analysis of digital gene expression data. Bioinformatics 26, 139-140.
3. Huber W, Carey VJ, Gentleman R, Anders S, Carlson M, Carvalho BS, Bravo HC, Davis S, Gatto L, Girke T, Gottardo R, Hahne F, Hansen KD, Irizarry RA, Lawrence M, Love MI, MacDonald J, Obenchain V, Oleś AK, Pagès H, Reyes A, Shannon P, Smyth GK, Tenenbaum D, Waldron L, Morgan M. Orchestrating high-throughput genomic analysis with Bioconductor. Nat Methods. 2015 Feb;12(2):115-21.
4. R Core Team (2016). R: A language and environment for statistical computing. R Foundation for Statistical Computing, Vienna, Austria. URL https://www.R-project.org/.

[Table of Contents](#Table-of-Contents)

## Appendix: R Session Info

In [39]:
Sys.time()
sessionInfo()

[1] "2020-02-13 12:11:23 PST"

R version 3.5.2 (2018-12-20)
Platform: x86_64-apple-darwin15.6.0 (64-bit)
Running under: macOS Mojave 10.14.6

Matrix products: default
BLAS: /Library/Frameworks/R.framework/Versions/3.5/Resources/lib/libRblas.0.dylib
LAPACK: /Library/Frameworks/R.framework/Versions/3.5/Resources/lib/libRlapack.dylib

locale:
[1] en_US.UTF-8/en_US.UTF-8/en_US.UTF-8/C/en_US.UTF-8/en_US.UTF-8

attached base packages:
[1] parallel  stats4    stats     graphics  grDevices utils     datasets 
[8] methods   base     

other attached packages:
 [1] edgeR_3.24.3                           
 [2] limma_3.38.3                           
 [3] Homo.sapiens_1.3.1                     
 [4] TxDb.Hsapiens.UCSC.hg19.knownGene_3.2.2
 [5] org.Hs.eg.db_3.7.0                     
 [6] GO.db_3.7.0                            
 [7] OrganismDbi_1.24.0                     
 [8] GenomicFeatures_1.34.8                 
 [9] GenomicRanges_1.34.0                   
[10] GenomeInfoDb_1.18.2                    
[11] AnnotationDbi_1.44.

[Table of Contents](#Table-of-Contents)

Copyright (c) 2018 UC San Diego Center for Computational Biology & Bioinformatics under the MIT License

Notebook template by Amanda Birmingham